In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from itertools import combinations
from scipy import stats
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import math

In [6]:
#import test and train datasets

In [26]:
df_train = pd.read_csv('/Users/lakshikajain/Documents/GitHub/DataScienceCapstone_2/output.csv')
df_test = pd.read_csv('/Users/lakshikajain/Documents/GitHub/DataScienceCapstone_2/Testing.csv')

In [8]:
#dropped Fluid_overload as all rows for this field is 0

In [27]:
df_train.drop('fluid_overload', axis = 1, inplace = True)

In [28]:
df_train_2 = df_train.loc[:, df_train.columns != "prognosis"]

In [29]:
df_train_2 = df_train_2.apply(pd.to_numeric, errors = 'coerce')

In [30]:
df_train = pd.concat([df_train_2,df_train['prognosis']], axis=1, join='inner')

In [31]:
# Fit the model

In [32]:
X = df_train.drop('prognosis', axis = 1)
y = df_train['prognosis'].values
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

array([[ 2.50132708,  2.29336913,  6.6749948 , -0.2173802 , -0.14981285,
        -0.43999471, -0.40183714, -0.2173802 , -0.2173802 , -0.14981285,
        -0.14981285, -0.79795147, -0.21428571, -0.14981285, -0.80410593,
        -0.15401412, -0.15401412, -0.15401412, -0.22043899, -0.31960998,
        -0.22043899, -0.31960998, -0.14981285, -0.15401412, -0.35982855,
        -0.61870779, -0.14981285, -0.31728716, -0.39978778, -0.14981285,
        -0.2173802 , -0.54728837, -0.47701668, -0.36198685, -0.55105045,
        -0.55293081, -0.15811388, -0.22043899, -0.22043899, -0.51520103,
        -0.35982855, -0.27844132, -0.15401412, -0.44590404, -0.15401412,
        -0.15401412, -0.27589035, -0.40795783, -0.27332236, -0.27844132,
        -0.15811388, -0.15811388, -0.15811388, -0.15811388, -0.15811388,
        -0.40592207, -0.14981285, -0.2234636 , -0.15401412, -0.15401412,
        -0.15401412, -0.15401412, -0.22043899, -0.27073683, -0.15401412,
        -0.15401412, -0.22043899, -0.15401412, -0.1

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.01, random_state = 0)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(4870, 131) (4870,)
(50, 131) (50,)


In [34]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

# Separate features and target
X = df_train.drop(columns=['prognosis'])
y = df_train['prognosis']

# Encode the target variable if it's categorical
le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Initialize models
models = {
    'RandomForest': RandomForestClassifier(random_state=42),
    'DecisionTree': DecisionTreeClassifier(random_state=42),
    'MLP': MLPClassifier(max_iter=300, random_state=42),
    'XGBoost': xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
}

# Dictionary to store the AUC scores
model_results = {}

# Train and evaluate each model
for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict probabilities for the test set
    y_prob = model.predict_proba(X_test)
    
    # Calculate AUC score for the multi-class problem using 'ovr' (One-vs-Rest)
    auc_score = roc_auc_score(y_test, y_prob, multi_class='ovr')
    
    # Store the result
    model_results[model_name] = auc_score

# Summarize the results
model_results



/opt/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [20:32:27] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


{'RandomForest': 1.0, 'DecisionTree': 1.0, 'MLP': 1.0, 'XGBoost': 1.0}

In [36]:
df_test.drop('fluid_overload', axis = 1, inplace = True)

In [38]:
# Redefine the align_predictions function
def align_predictions(y_prob, num_classes=41):
    if y_prob.shape[1] < num_classes:
        # Create an array of zeros for missing class probabilities
        missing_classes = num_classes - y_prob.shape[1]
        y_prob_aligned = np.hstack((y_prob, np.zeros((y_prob.shape[0], missing_classes))))
        return y_prob_aligned
    return y_prob

# Retry with previously defined models except MLP
models_retrain = {
    'RandomForest': RandomForestClassifier(random_state=42),
    'DecisionTree': DecisionTreeClassifier(random_state=42),
    'XGBoost': xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
}

# Re-train the models
for model_name, model in models_retrain.items():
    model.fit(X_train, y_train)

# Re-evaluate the models on the new test dataset
test_results = {}

for model_name, model in models_retrain.items():
    # Predict probabilities for the test set
    y_prob = model.predict_proba(X_test_new)
    
    # Align probabilities with all 41 classes
    y_prob_aligned = align_predictions(y_prob)
    
    # Calculate AUC score for the multi-class problem
    auc_score = roc_auc_score(y_test_new_encoded, y_prob_aligned, multi_class='ovr')
    
    # Store the result
    test_results[model_name] = auc_score

# Display the test results
test_results


/opt/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [20:40:56] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


{'RandomForest': 1.0, 'DecisionTree': 0.9936049970255799, 'XGBoost': 1.0}